In [17]:
import pandas as pd
import json 

In [18]:
with open("../input_data/results.json") as file_to_open:
    data = file_to_open.read()
json_data = json.loads(data)

In [19]:
print(json_data.keys())

dict_keys(['meta', 'files', 'summary', 'packages'])


## Get Packages Data

CSV Schema for vulnerability data:
|id |title|description|cwe|cve|severity|epss.cve|epss.epss|epss.percentile|epss.date|
|---|-----|-----------|---|---|--------|--------|---------|---------------|---------|
|   |     |           |   |   |        |        |         |               |         |


In [20]:
package_vulnerability_raw_data = json_data.get("packages")
vulnerability_to_df = pd.DataFrame(package_vulnerability_raw_data)
vulnerability_df_filtered = vulnerability_to_df.map(lambda x: x[0] if isinstance(x, list) else x)

In [21]:
# normalized_coordinates_data = vulnerability_df_filtered['coordinates']
# normalized_vulnerability_data = pd.json_normalize(vulnerability_df_filtered['vulnerabilities'])

In [22]:
normalized_vulnerability_data = pd.json_normalize(package_vulnerability_raw_data)

In [23]:
df_dict = pd.json_normalize(normalized_vulnerability_data['vulnerabilities'].explode()).reset_index()

In [24]:
df_final = normalized_vulnerability_data.drop('vulnerabilities', axis=1).join(df_dict.set_index('index'))

In [25]:
df_final

,coordinates,id,title,description,cwe,cve,severity,epss.cve,epss.epss,epss.percentile,epss.date
0,pkg:nuget/Microsoft.Rest.ClientRuntime@2.3.23,CVE-2022-26907,Azure SDK for .NET Information Disclosure Vuln...,Azure SDK for .NET Information Disclosure Vuln...,CWE-532,CVE-2022-26907,MODERATE,CVE-2022-26907,0.005440000,0.747700000,2023-11-22
1,pkg:nuget/Newtonsoft.Json@11.0.2,CWE-755,Improper Handling of Exceptional Conditions in...,Newtonsoft.Json prior to version 13.0.1 is vul...,CWE-755,NaN,HIGH,NaN,NaN,NaN,NaN
2,pkg:nuget/System.Text.Encodings.Web@4.5.0,"BIT-2021-26701,BIT-dotnet-2021-26701,BIT-dotne...",.NET Core Remote Code Execution Vulnerability,.NET Core Remote Code Execution Vulnerability ...,NaN,"BIT-2021-26701,BIT-dotnet-2021-26701,BIT-dotne...",CRITICAL,NaN,NaN,NaN,NaN
3,pkg:nuget/System.Text.RegularExpressions@4.3.0,CVE-2019-0820,Regular Expression Denial of Service in System...,A denial of service vulnerability exists when ...,"CWE-1333,CWE-400",CVE-2019-0820,HIGH,CVE-2019-0820,0.001590000,0.523850000,2023-11-22
4,pkg:nuget/Newtonsoft.Json@9.0.1,CWE-755,Improper Handling of Exceptional Conditions in...,Newtonsoft.Json prior to version 13.0.1 is vul...,CWE-755,NaN,HIGH,NaN,NaN,NaN,NaN
5,pkg:nuget/System.Text.Encodings.Web@4.7.1,"BIT-2021-26701,BIT-dotnet-2021-26701,BIT-dotne...",.NET Core Remote Code Execution Vulnerability,.NET Core Remote Code Execution Vulnerability ...,NaN,"BIT-2021-26701,BIT-dotnet-2021-26701,BIT-dotne...",CRITICAL,NaN,NaN,NaN,NaN
6,pkg:nuget/Newtonsoft.Json@10.0.2,CWE-755,Improper Handling of Exceptional Conditions in...,Newtonsoft.Json prior to version 13.0.1 is vul...,CWE-755,NaN,HIGH,NaN,NaN,NaN,NaN
7,pkg:nuget/System.Net.Http@4.3.0,CVE-2018-8292,.NET Core Information Disclosure,An information disclosure vulnerability exists...,CWE-200,CVE-2018-8292,HIGH,CVE-2018-8292,0.025470000,0.890510000,2023-11-22
8,pkg:nuget/Azure.Identity@1.8.2,CVE-2023-36414,Azure Identity SDK Remote Code Execution Vulne...,Azure Identity SDK is vulnerable to remote cod...,NaN,CVE-2023-36414,HIGH,CVE-2023-36414,0.001890000,0.563620000,2023-11-22
9,pkg:nuget/System.Net.Http@4.1.0,CVE-2018-8292,.NET Core Information Disclosure,An information disclosure vulnerability exists...,CWE-200,CVE-2018-8292,HIGH,CVE-2018-8292,0.025470000,0.890510000,2023-11-22


In [26]:
df_final['componentname']=df_final['coordinates'].str.extract(r'pkg:[a-zA-Z]*/([a-zA-Z.]*)@[0-9.]*')


## Import component location data

In [27]:
with open("../input_data/component_by_file.json") as file_to_open:
    data = file_to_open.read()
json_data = json.loads(data)

In [28]:
json_data.keys()

dict_keys(['Corvus.UriTemplates.sbom-20231024155652770.json', 'Corvus.AzureFunctionsKeepAlive.sbom-20230731145443527.json', 'Corvus.Retry.sbom-20230802021549215.json', 'Marain.Claims.sbom-20230925152701828.json', 'Corvus.Extensions.sbom-20230927021217870.json', 'Menes.sbom-20230927023952519.json', 'Corvus.ContentHandling.sbom-20230927021312061.json', 'Marain.Tenancy.sbom-20230922163414724.json', 'Marain.TenantManagement.sbom-20230927082242823.json', 'Corvus.Globbing.sbom-20230925021641016.json', 'Marain.Workflow.sbom-20231018030131154.json', 'Corvus.AzureFunctionsKeepAlive.sbom-20230926005207504.json', 'Corvus.Globbing.sbom-20230927022310790.json', 'Corvus.Json.sbom-20230802021609249.json', 'Marain.Operations.sbom-20230925121354442.json', 'Corvus.ContentHandling.sbom-20230803021854521.json', 'Corvus.Monitoring.sbom-20230927021528451.json', 'Corvus.ContentHandling.sbom-20230927022958777.json', 'Corvus.Extensions.System.Text.Json.sbom-20230731141202223.json', 'Corvus.Extensions.sbom-2023

In [29]:
test = pd.DataFrame.from_dict(json_data,orient='index')

In [30]:
# Reset the index to make it a column
test.reset_index(inplace=True)

# Rename the columns
test.columns = ['sbom_name'] + list(test.columns[1:])

In [31]:
df_melted = test.melt(id_vars='sbom_name', value_vars=test.columns[1:], value_name='componentname')

In [32]:
# Drop the 'Variable' column
df_melted.drop('variable', axis=1, inplace=True)

# Drop rows with 'None' values
df_melted.dropna(inplace=True)

df_melted.drop_duplicates(inplace=True)

df_melted.sort_values(by='sbom_name', inplace=True)

df_melted = df_melted.reset_index(drop=True)

## Merge tables

In [33]:
df_melted

,sbom_name,componentname
0,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...,System.Memory.Data
1,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...,Microsoft.AspNetCore.Mvc.WebApiCompatShim
2,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...,Microsoft.AspNetCore.Hosting.Server.Abstractions
3,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...,System.Runtime.Serialization.Primitives
4,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...,System.Configuration.ConfigurationManager
...,...,...
11523,Menes.sbom-20231019081033177.json,coverlet.msbuild
11524,Menes.sbom-20231019081033177.json,System.Threading.ThreadPool
11525,Menes.sbom-20231019081033177.json,Microsoft.Extensions.FileProviders.Physical
11526,Menes.sbom-20231019081033177.json,System.Xml.XPath


In [34]:
join_table = df_final.merge(df_melted, on='componentname', how='left')

In [35]:
join_table.to_csv('../results/vulnerability_report.csv')

## Get Summary Data

In [36]:
summary_raw_data = json_data.get("summary")
summary_to_df = pd.DataFrame(summary_raw_data, index=[0])

In [37]:
summary_to_df

""
0


In [38]:
summary_to_df.to_csv('../results/summary_report.csv')